<a href="https://colab.research.google.com/github/Gaju27/EVA/blob/master/eva4_session4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1), # image 28x28 with padding 1 now image size 30x30
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 8, 3, padding=1), # after convolve now again image size 28x28 padding=1 becomes 30x30
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 8, 3, stride=2, padding=1), # after convolve now again image size 14x14 padding=1 becomes 16x16 with stride=2 is move two pixels at time here
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2, 2), # max pooling becomes 7x7
            nn.Dropout(0.25) 
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1), # input image size 7x7 with 16 kernels and padding=1 now image size 9x9
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1), # after convolve now again image size 7x7 padding=1 becomes 9x9
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, stride=2, padding=1), # after convolve now again image size 4x4 padding=1 becomes 6x6 with stride=2 is move two pixels at time here
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2), # max pooling becomes 2x2
            nn.Dropout(0.25) 
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1), # input image size 2x2 with 32 kernels and padding=1 now image size 4x4
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # max pooling becomes 1x1
            nn.Dropout(0.25) 
        )
        
#         self.fc = nn.Sequential(
#             nn.Linear(128, 10)

        self.avg_pool1=nn.AvgPool2d(kernel_size=3, stride=1, padding=1)
        
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = self.conv3(x)
        
        x = self.avg_pool1(x) # Taken average: GAP layer
        x = x.view(x.size(0), -1)
#         x = self.fc(x)
        
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4            [-1, 8, 28, 28]             584
              ReLU-5            [-1, 8, 28, 28]               0
       BatchNorm2d-6            [-1, 8, 28, 28]              16
            Conv2d-7            [-1, 8, 14, 14]             584
              ReLU-8            [-1, 8, 14, 14]               0
       BatchNorm2d-9            [-1, 8, 14, 14]              16
        MaxPool2d-10              [-1, 8, 7, 7]               0
          Dropout-11              [-1, 8, 7, 7]               0
           Conv2d-12             [-1, 16, 7, 7]           1,168
             ReLU-13             [-1, 16, 7, 7]               0
      BatchNorm2d-14             [-1, 1

Total paramaters: 11,840

In [4]:


torch.manual_seed(1)
batch_size = 16 # changed batch_size from 128 to 16 it performed better

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.00 * float(correct) / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16): #We have changed epoch 16.
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.1734, Accuracy: 9734/10000 (97.34%)



  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0890, Accuracy: 9867/10000 (98.67%)



  0%|          | 0/3750 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0530, Accuracy: 9907/10000 (99.07%)



  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0510, Accuracy: 9915/10000 (99.15%)



loss=1.0914143323898315 batch_id=3749: 100%|██████████| 3750/3750 [00:40<00:00, 91.47it/s]



Test set: Average loss: 0.0450, Accuracy: 9919/10000 (99.19%)

